# Part 2 - Visualizing a Decision Tree - https://youtu.be/tNa99PG8hR8

Build one on a real dataset, add code to visualize it, and practice reading it - so 
you can see how it works under the hood.

Use Iris flower data set: https://en.wikipedia.org/wiki/Iris_flower_data_set
Identify type of flower based on measurements
Dataset includes 3 species of Iris flowers: setosa, versicolor, virginica
4 features used to describe: length and width of sepal and petal
50 examples of each type for 150 total examples

Goals
 
 1. Import dataset
 2. Train a classifier
 3. Predict label for new flower
 4. Visualize the tree

scikit-learn datasets: http://scikit-learn.org/stable/datasets/
already includes Iris dataset: `load_iris`

In [5]:
# import endo
# endo.l = locals()

from sklearn.datasets import load_iris
from sklearn import tree
import pandas as pd  # 只是用來查看 iris 
import numpy as np
import peforth

iris = load_iris()

In [6]:
# print entire dataset
# for i in range(len(iris.target)):
#     print ("Example %d: label %s, features %s" % (i, iris.target[i], iris.data[i]))

### jupyter notebook 有更好的方式查看 dataset
以下先把它轉成 Pandas's data-frame 然後直接查看之，漂亮多了。

In [7]:
%%f 
__main__ :> iris constant iris // ( -- dataset )
iris type tib. cr
iris dir tib. cr
iris :> feature_names tib. cr
iris :> target_names tib. cr    

iris type tib. \ ==> <class 'sklearn.utils.Bunch'> (<class 'type'>)

iris dir tib. \ ==> ['DESCR', 'data', 'feature_names', 'target', 'target_names'] (<class 'list'>)

iris :> feature_names tib. \ ==> ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'] (<class 'list'>)

iris :> target_names tib. \ ==> ['setosa' 'versicolor' 'virginica'] (<class 'numpy.ndarray'>)



In [8]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
5,5.4,3.9,1.7,0.4,0.0
6,4.6,3.4,1.4,0.3,0.0
7,5.0,3.4,1.5,0.2,0.0
8,4.4,2.9,1.4,0.2,0.0
9,4.9,3.1,1.5,0.1,0.0


    ^^^^^^^ Iris dataset 看清楚了 ^^^^^^^^^^^^

In [9]:
# Testing Data
# Examples used to test the classifier's accuracy
# Not part of the training data
# here, we remove the first example of each flower
# found at indices: 0, 50, 100
test_idx = [0, 50, 100]  # 故意挑出三個出來，當作測試組。每種類各挑一個，共三個。其餘的留在訓練組。

# create 2 new sets of variables, for training and testing
# training data
# remove the entires from the data and target variables
train_target = np.delete(iris.target, test_idx)       # 從訓練組中剔除測試組的三筆資料
train_data = np.delete(iris.data, test_idx, axis=0)

### 研究一下 np.delete() 幹嘛的？

In [10]:
# This cell creates %%csv cell magic. See http://ipython-books.github.io/14-creating-an-ipython-extension-with-custom-magic-commands
import pandas as pd
from io import StringIO
from IPython.core.magic import (register_cell_magic)
@register_cell_magic
def csv(line, cell):
    return pd.read_csv(StringIO(cell), sep=line)

In [11]:
%%csv \s+
a b c
1 2 3
4 5 6
7 8 9
11 22 33
44 55 66
77 88 99

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9
3,11,22,33
4,44,55,66
5,77,88,99


In [12]:
df = _

In [13]:
np.delete(np.array(df),[1,3,5],axis=0)

array([[ 1,  2,  3],
       [ 7,  8,  9],
       [44, 55, 66]], dtype=int64)

    ^^^^^^^^^^ np.delete() 搞懂了 ^^^^^^^^^^^^^^^

In [14]:
# testing data
test_target = iris.target[test_idx]  # 故意挑出三個出來，當作測試組。每種類各挑一個，共三個。其餘的留在訓練組。
test_data = iris.data[test_idx]

In [15]:
# create new classifier
clf = tree.DecisionTreeClassifier() # 祭出法寶
# train on training data
clf.fit(train_data, train_target) # 施咒，請寶貝轉身！

# what we expect
# print test_target

#Python 3.6.0 |Anaconda 4.3.1
print(test_target)

# what tree predicts
#print clf.predict(test_data)

#Python 3.6.0 |Anaconda 4.3.1
print (clf.predict(test_data))

# Visualize
# from scikit decision tree tutorial: http://scikit-learn.org/stable/modules/tree.html
from sklearn.externals.six import StringIO
import pydotplus as pydot

dot_data = StringIO()
tree.export_graphviz(clf, out_file=dot_data,
                     feature_names=iris.feature_names,
                     class_names=iris.target_names,
                     filled=True, rounded=True,
                     impurity=False)
graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("iris.pdf")

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

[0 1 2]
[0 1 2]


True

### graph.write_pdf("iris.pdf") 有問題

別人已經問了： https://stackoverflow.com/questions/38176472/graph-write-pdfiris-pdf-attributeerror-list-object-has-no-attribute-writ

答案是：

1. 要改用 `pydotplus` instead of `pydot`
2. _pydotplus.graphviz.InvocationException: GraphViz's executables not found_<br>
   https://stackoverflow.com/questions/18438997/why-is-pydot-unable-to-find-graphvizs-executables-in-windows-8<br>
   https://stackoverflow.com/questions/28312534/graphvizs-executables-are-not-found-python-3-4<br>
   要 install Graphviz<br>
   `   c:\\Users\hcche\Downloads\windows _ Graphviz - Graph Visualization Software.pdf`<br>
   `   c:\\Users\hcche\Downloads\graphviz-2.38.msi`
3. 要為 Graphviz 設置 PATH
   Add PATH manually --> `path=%PATH%c:\Program Files (x86)\Graphviz2.38\bin` (T550 WKS-4AEN0404)

成功了！

